In [ ]:
#importing libraries
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
import cv2
import numpy as np
import os
import random
import pandas as pd
import glob
import matplotlib.pyplot as plt


In [ ]:
#define model class

from torch import nn
from torchvision import models
class MyModel(nn.Module):
      def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(MyModel, self).__init__()
        model = models.resnext50_32x4d(pretrained = True) #Residual Network CNN
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
      def forward(self, x):
          batch_size,seq_length, c, h, w = x.shape
          x = x.view(batch_size * seq_length, c, h, w)
          fmap = self.model(x)
          x = self.avgpool(fmap)
          x = x.view(batch_size,seq_length,2048)
          x_lstm,_ = self.lstm(x,None)
          return fmap,self.dp(self.linear1(torch.mean(x_lstm,dim = 1)))


model = MyModel(2)

device=torch.device('cpu')
PATH="/content/drive/MyDrive/Colab Notebooks/Deepfake Detection Project/files/model(1).pth"
model.load_state_dict(torch.load(PATH,map_location=device))
model.eval()

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:00<00:00, 121MB/s]


MyModel(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (

Upload a video

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving 22_24__walking_and_outside_surprised__XL557XC6.mp4 to 22_24__walking_and_outside_surprised__XL557XC6.mp4


In [ ]:
!pip install dlib

!pip install face_recognition


In [ ]:
import cv2
import dlib
import os
from tqdm import tqdm


def frame_extract(path):
  vidObj = cv2.VideoCapture(path)
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image


def create_face_video(input_video_path, output_video_path):
    frames = []
    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 30, (112, 112))

    # Initialize face detector from dlib
    face_detector = dlib.get_frontal_face_detector()

    for idx, frame in enumerate(frame_extract(input_video_path)):
        if idx <= 150:
            frames.append(frame)
            if len(frames) == 4:
                # Convert frames to grayscale for face detection
                gray_frames = [cv2.cvtColor(f, cv2.COLOR_BGR2GRAY) for f in frames]

                # Detect faces using dlib face detector
                faces = [face_detector(gray_frame) for gray_frame in gray_frames]

                for i, face in enumerate(faces):
                    if face:
                        top, right, bottom, left = (face[0].top(), face[0].right(), face[0].bottom(), face[0].left())
                        try:
                            out.write(cv2.resize(frames[i][top:bottom, left:right, :], (112, 112)))
                        except:
                            pass
                frames = []

    out.release()


# Example usage:
input_video_path = "/content/22_24__walking_and_outside_surprised__XL557XC6.mp4"
output_video_path = "/content/video7.mp4"

create_face_video(input_video_path, output_video_path)


Preprocess video


In [ ]:
def get_data_transforms(im_size=112):
    mean = [0.485, 0.456, 0.406]
    std = [0.229, 0.224, 0.225]

    train_transforms = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((im_size, im_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    return train_transforms

In [ ]:
random_video_path = "/content/video7.mp4"
def preprocess(random_video_path):

    video_frames = []

    cap = cv2.VideoCapture(random_video_path)
    while True:
        success, frame = cap.read()
        if not success:
            break
        video_frames.append(frame)

    cap.release()

    # Preprocess the video frames
    data_transform = get_data_transforms()
    processed_frames = [data_transform(frame) for frame in video_frames]
    video_tensor = torch.stack(processed_frames)

    # Ensure that the sequence_length parameter matches the one used during training
    sequence_length = 60
    video_tensor = video_tensor[:sequence_length]
    return video_tensor

def predict(video_tensor,model):
    # Run inference
    if torch.cuda.is_available():
        video_tensor = video_tensor.cuda()

    predicted_labels = []

    with torch.no_grad():
        for frame_tensor in video_tensor:
            # Unsqueeze to add the batch and sequence dimensions
            frame_tensor = frame_tensor.unsqueeze(0).unsqueeze(1)
            _, predictions = model(frame_tensor)
            _, predicted_label = torch.max(predictions, 1)
            predicted_label = predicted_label.item()

            # Accumulate predicted labels
            predicted_labels.append(predicted_label)
            return predicted_labels

def output_predict(predicted_labels):
    # Calculate overall accuracy
    overall_accuracy = (predicted_labels.count(1) / len(predicted_labels)) * 100.0

    # Determine the overall predicted label
    overall_predicted_label = 1 if overall_accuracy > 50 else 0

    # Print or use the results
    print(f"Overall Accuracy: {overall_accuracy:.2f}%")
    print(f"Overall Prediction: {'REAL' if overall_predicted_label == 1 else 'FAKE'}")


Overall Accuracy: 93.33%
Overall Prediction: REAL


In [ ]:
video_tensor = preprocess(random_video_path)
predicted_labels = predict(video_tensor,model)
output_predict(predicted_labels)